In [3]:
import numpy as np
import pandas as pd
from scipy.io import loadmat, mmread
import sys
import itertools 
import pickle
import matplotlib.pylab as plt
import time

In [2]:
%matplotlib inline

In [4]:
import rankingmethod
import rankingmeasure
import rank

In [5]:
%load_ext autoreload
%autoreload 2

In [6]:
from  experiment import *

# loading data sets

In [7]:
data = pd.read_csv("../datasets/soc-Epinions1.txt", sep='\t', header=4)
data = data.as_matrix()

In [8]:
epinion, epinion_shape = make_valid_data(data)

In [9]:
data = pd.read_csv("../datasets/Slashdot0811.txt", sep='\t', header=4)
data = data.as_matrix()

In [10]:
slashdot, slashdot_shape = make_valid_data(data)

In [11]:
data =  pd.read_csv("../datasets/ml-100k/u.data", sep='\t', header=None).ix[:, :1]
data = data.as_matrix()

In [12]:
ml100k, ml100k_shape = make_valid_data(data)

In [13]:
data = pd.read_csv("../datasets/ml-1m/ratings.dat", sep='::', header=None,  engine='python').ix[:, :1]
data = data.as_matrix()

In [14]:
ml1m, ml1m_shape = make_valid_data(data)

### Зададим параметры, которые были выбраны при помощивалидации

In [15]:
def create_list_cf(user_item):
    list_cf = [
        rankingmethod.PopRec(user_item),
#         rankingmethod.CLiMF(user_item, K=10, reg=0.001, lrate=0.01, verbose=0, maxiter=10),
        
#         rank.CLiMF(user_item, K=5, reg=0.01, lrate=0.1, verbose=0, maxiter=10),
#         rank.CLiMF(user_item, K=5, reg=0.01, lrate=0.01, verbose=0, maxiter=10),
        rank.CLiMF(user_item, K=5, reg=0.01, lrate=0.001, verbose=0, maxiter=10),
#         rank.CLiMF(user_item, K=15, reg=0.001, lrate=0.1, verbose=0, maxiter=10),
        rank.BPR_MF(user_item, K = 20, lrate=0.012, regIpos=0.012, 
                                  regIneg=0.012, regU=0.012, maxiter=20, verbose=0 ),
        
#         rank.CLiMF(user_item, K=10, reg=0.001, lrate=0.1, verbose=0, maxiter=10),
#         rank.CLiMF(user_item, K=10, reg=0.001, lrate=0.001, verbose=0, maxiter=10),
#         rankingmethod.BPR_MF(user_item, K = 20, lrate=0.012, regIpos=0.012, 
#                                   regIneg=0.012, regU=0.012, maxiter=4, verbose=0 ),
        rankingmethod.iMF(user_item, K=10, lmbd=0.01, alpha=40, maxiter=30, verbose=0),
        rankingmethod.TFMAP(user_item, lrate=0.01, maxiter=10),
#         rank.TFMAP(user_item, lrate=0.01, maxiter=20)
    ]
    return list_cf#[list_cf[i] for i in index]

### Проведение такого же эксперимента, как у создатей CLiMF(Точнее похожего, т.к. было решено не удалять топ 3 всех предметов. И даже с этим удалением получить такие же цифры, что авторы epinion не получилось)

In [16]:
def climf_experiment(data, list_cf, K=5, maxiter=1, topK=5, verbose=0, validation_type=1):
    res = np.zeros((6, len(list_cf)))
    for t in range(maxiter):
        if validation_type == 0:
            test, train, traintest = givenK_train_test(data, K)
        elif validation_type == 1:
            test, train, traintest = ratio_train_test(data, K)
        for i, cf in enumerate(list_cf):
            start_time = time.time()
            cf.fit(train)
#             print(time.time() - start_time)
            prec_k, one_recal_k, ngdc, mp = rankingmeasure.get_Kmetrics(train, test, cf, K=topK)
            mrr, auc = rankingmeasure.get_listmetrics(train, test, cf)
            res[:, i] += np.array([prec_k, one_recal_k, ngdc, mp, mrr, auc])
            if verbose == 1:
                print(t, i, res[:, i] / (t + 1))
                sys.stdout.flush()
    res /= maxiter
    res = pd.DataFrame(res)
    
    res.columns = [type(cf).__name__ for cf in list_cf]
    res.index = [name + str(topK) for name in ['prec@', '1recal@', 'NDCG@', 'MAP@']] + ['MRR', 'AUC']
#     res.columns = ['PopRec', 'CLiMF', 'BRP_MP', 'iMF', 'TFMAP']
#     res.index = ['prec@' + str(topK),'1recal@' + str(topK), 'NDCG@' + str(topK), 
#                'MAP@' + str(topK), 'MRR', 'AUC']
    return res

In [18]:
climf_experiment(ml100k, create_list_cf(ml100k_shape), K =0.1, maxiter=5, topK=5,  verbose=0)

,PopRec,CLiMF,BPR_MF,iMF,TFMAP
prec@5,0.141290,0.140248,0.143474,0.152010,0.083573
1recal@5,0.472705,0.455831,0.497270,0.535732,0.327792
NDCG@5,0.154562,0.150959,0.153422,0.156756,0.092821
MAP@5,0.288754,0.273045,0.289482,0.289339,0.192158
MRR,0.338996,0.319950,0.332253,0.340819,0.233657
AUC,0.856632,0.814014,0.785816,0.927055,0.853811


In [19]:
climf_experiment(ml100k, create_list_cf(ml100k_shape), K =0.2, maxiter=5, topK=5,  verbose=0)

,PopRec,CLiMF,BPR_MF,iMF,TFMAP
prec@5,0.237022,0.236675,0.238263,0.259653,0.165955
1recal@5,0.643672,0.627295,0.669231,0.737717,0.540447
NDCG@5,0.257089,0.254108,0.249028,0.264853,0.172586
MAP@5,0.419890,0.408746,0.409753,0.427131,0.304675
MRR,0.479092,0.459178,0.453254,0.474468,0.353326
AUC,0.857531,0.806631,0.779713,0.926793,0.855633


In [20]:
climf_experiment(epinion, create_list_cf(epinion_shape), K =0.1, maxiter=3, topK=5,  verbose=0)

,PopRec,CLiMF,BPR_MF,iMF,TFMAP
prec@5,0.030193,0.029406,0.009247,0.069406,0.027317
1recal@5,0.132728,0.130004,0.044873,0.272645,0.121302
NDCG@5,0.033896,0.033200,0.013033,0.074202,0.031213
MAP@5,0.076760,0.075752,0.035300,0.151523,0.071640
MRR,0.099977,0.099058,0.043925,0.186433,0.094672
AUC,0.869080,0.768566,0.690515,0.917501,0.860799


In [21]:
climf_experiment(epinion, create_list_cf(epinion_shape), K =0.2, maxiter=3, topK=5,  verbose=1)

0 0 [ 0.05511918  0.22451759  0.06210466  0.13409509  0.16639616  0.86979699]
0 1 [ 0.05334847  0.21861521  0.06072931  0.13216988  0.16502904  0.75594446]
0 2 [ 0.02265607  0.10419977  0.02899874  0.07293322  0.08808791  0.68817976]
0 3 [ 0.12258797  0.41611805  0.12859505  0.23681423  0.27869498  0.91656811]
0 4 [ 0.05684449  0.22633371  0.06371018  0.13507977  0.16642095  0.8613903 ]
1 0 [ 0.05650397  0.22973893  0.06386653  0.13810726  0.17089435  0.87021311]
1 1 [ 0.05489217  0.22519864  0.0622986   0.13559544  0.16845355  0.75728748]
1 2 [ 0.02088536  0.0969353   0.02765124  0.07117811  0.08629086  0.68679139]
1 3 [ 0.12426788  0.42167991  0.13103589  0.24212464  0.28381307  0.91582914]
1 4 [ 0.05611805  0.22644722  0.06369695  0.13726132  0.16921876  0.86166393]
2 0 [ 0.05643587  0.23019296  0.0636202   0.1375906   0.17039428  0.8700745 ]
2 1 [ 0.05454408  0.22376088  0.06199168  0.13530784  0.16806354  0.7576185 ]
2 2 [ 0.01967461  0.09201665  0.02677968  0.07009806  0.08406311

,PopRec,CLiMF,BPR_MF,iMF,TFMAP
prec@5,0.056436,0.054544,0.019675,0.124888,0.054862
1recal@5,0.230193,0.223761,0.092017,0.425123,0.222172
NDCG@5,0.063620,0.061992,0.026780,0.131694,0.062803
MAP@5,0.137591,0.135308,0.070098,0.243819,0.136352
MRR,0.170394,0.168064,0.084063,0.285330,0.168717
AUC,0.870074,0.757619,0.687072,0.915926,0.861527


In [22]:
climf_experiment(slashdot, create_list_cf(slashdot_shape), K=0.1, maxiter=3, topK=5,  verbose=0)

,PopRec,CLiMF,BPR_MF,iMF,TFMAP
prec@5,0.015494,0.015475,0.008600,0.039350,0.013987
1recal@5,0.072178,0.072511,0.040952,0.158276,0.066219
NDCG@5,0.017996,0.017908,0.010541,0.043010,0.014496
MAP@5,0.042828,0.042703,0.026310,0.090153,0.032572
MRR,0.059717,0.058998,0.033803,0.115116,0.049009
AUC,0.845791,0.721384,0.649001,0.870335,0.837646


In [ ]:
climf_experiment(slashdot, create_list_cf(slashdot_shape), K=0.2, maxiter=3, topK=5,  verbose=0)

,PopRec,CLiMF,BPR_MF,iMF,TFMAP
prec@5,0.029434,0.029567,0.015418,0.072235,0.028986
1recal@5,0.130768,0.132294,0.071558,0.262681,0.129529
NDCG@5,0.033527,0.033479,0.018743,0.078109,0.032499
MAP@5,0.076428,0.076449,0.045782,0.152282,0.073827
MRR,0.102440,0.101305,0.057159,0.186481,0.098932
AUC,0.846083,0.707364,0.648265,0.866559,0.835498


In [17]:
climf_experiment(ml1m, create_list_cf(ml1m_shape), K=0.1, maxiter=3, topK=5, verbose=1)

0 0 [ 0.13779059  0.42295909  0.14780291  0.26060201  0.301097    0.85815545]
0 1 [ 0.13137502  0.40782123  0.14050909  0.24860611  0.2922643   0.85393526]
0 2 [ 0.09039467  0.29572896  0.09982597  0.18713432  0.22594189  0.74041612]
0 3 [ 0.1650027   0.53793476  0.17085658  0.30235603  0.34899273  0.9237604 ]
0 4 [ 0.04541359  0.20291944  0.04666301  0.09973268  0.12967069  0.83891775]
1 0 [ 0.13656515  0.42196792  0.14681176  0.26039639  0.30141503  0.8582571 ]
1 1 [ 0.13184358  0.41034421  0.14028065  0.24808549  0.29127163  0.85401047]
1 2 [ 0.09160209  0.30329789  0.10057758  0.18844074  0.22536154  0.73876547]
1 3 [ 0.16485853  0.53685349  0.17134657  0.30408757  0.35154562  0.92420531]
1 4 [ 0.04586412  0.2066138   0.0510389   0.11588274  0.14781736  0.83884389]
2 0 [ 0.13714183  0.42163753  0.14732632  0.26033311  0.30185296  0.85818784]
2 1 [ 0.1320358   0.41202619  0.14022011  0.2478221   0.29093732  0.85392556]
2 2 [ 0.09240103  0.30552051  0.1011499   0.1886617   0.2251103 

,PopRec,CLiMF,BPR_MF,iMF,TFMAP
prec@5,0.137142,0.132036,0.092401,0.164594,0.040211
1recal@5,0.421638,0.412026,0.305521,0.534991,0.183456
NDCG@5,0.147326,0.140220,0.101150,0.170794,0.044135
MAP@5,0.260333,0.247822,0.188662,0.302587,0.100207
MRR,0.301853,0.290937,0.225110,0.349801,0.134565
AUC,0.858188,0.853926,0.736862,0.924362,0.838477


In [18]:
climf_experiment(ml1m, create_list_cf(ml1m_shape), K=0.2, maxiter=3, topK=5, verbose=0)

,PopRec,CLiMF,BPR_MF,iMF,TFMAP
prec@5,0.226083,0.216616,0.153986,0.278717,0.076987
1recal@5,0.568751,0.566829,0.430228,0.713522,0.317835
NDCG@5,0.237163,0.225544,0.164639,0.286774,0.085750
MAP@5,0.371927,0.356648,0.275540,0.440588,0.184778
MRR,0.413433,0.402515,0.315132,0.489345,0.223519
AUC,0.858286,0.854011,0.730523,0.923886,0.839656


### Построение ансамбля

# Netflix ensemble
# Regression

In [20]:
from sklearn import cross_validation
import rankensemble
import rankingmeasure

In [21]:
def netflix_ensemble_experiment(data, list_cf, list_ensemble, Ktrain=0.1, Kvalidation=0.1, 
                                     maxiter=1, topK=5,  verbose=0):
    res = np.zeros((4, len(list_cf) + len(list_ensemble)))
    for t in range(maxiter):
        test, train, traintest = ratio_train_test(data, Ktrain + Kvalidation)
        validation, train, trainvalidation = ratio_train_test(train, Kvalidation, data_is_xlist=True)
        
        for i in range(len(list_cf)):
            list_cf[i].fit(train)
            prec_k, one_recal_k, ngdc, mp = rankingmeasure.get_Kmetrics(trainvalidation, test, list_cf[i], K=topK)
            res[:, i] += np.array([prec_k, one_recal_k, ngdc, mp])
            if verbose == 1:
                print(t, i, res[:, i] / (t + 1))
                sys.stdout.flush()
        for j, ens in enumerate(list_ensemble):
            m = j + len(list_cf)
            ens.fit(list_cf, validation, train, trainvalidation)
#             return X, y
            prec_k, one_recal_k, ngdc, mp = rankingmeasure.get_Kmetrics(trainvalidation, test, ens, K=topK)
            res[:, m] += np.array([prec_k, one_recal_k, ngdc, mp])
            if verbose == 1:
                print(t,  m, res[:, m] / (t + 1))
#         for j, ens in enumerate(list_ensemble):
#             m = j + len(list_cf)
#             ens.fit(list_cf)
#             prec_k, one_recal_k, ngdc, mp = rankingmeasure.get_Kmetrics(train, test, ens, K=topK)
#             res[:, m] += np.array([prec_k, one_recal_k, ngdc, mp])
#             if verbose == 1:
#                 print(t, m, res[:, m] / (t +1))
        
    res /= maxiter
    res[:, len(list_cf) - 1] = np.max(res[:, :len(list_cf)], axis=1)
    res = res[:, len(list_cf)- 1:]
    res = pd.DataFrame(res)
    res.columns = ["best_result"] + [type(ens).__name__ for ens  in list_ensemble]
    res.index = [name + str(topK) for name in ['prec@', '1recal@', 'NDCG@', 'MAP@']]
#     res.index = []
#     res.columns = ['PopRec', 'CLiMF', 'BRP_MP', 'iMF', 'TFMAP', 'top_ensemble', 'rating_ensamble']
#     res.index = ['prec@' + str(topK),'1recal@' + str(topK), 'NDCG@' + str(topK), 
#                'MAP@' + str(topK)]
    return res

In [22]:
from sklearn import linear_model
import rankensemble
from sklearn import ensemble
from sklearn.kernel_ridge import KernelRidge

In [23]:
import copy

In [26]:
list_ensemble = [
    rankensemble.regression_ensemble(linear_model.Ridge(alpha=100)),
    rankensemble.inner_value_ensemble(),
    rankensemble.boost_val_ens(rankingmeasure.get_NDCG,[3,0,1,2], num_weights=11),
    rankensemble.tree_val_ens(rankingmeasure.get_NDCG, [[3, 0],[1,2]], num_weights=11)
]

In [28]:
netflix_ensemble_experiment(ml100k, create_list_cf(ml100k_shape)[1:5], list_ensemble, Ktrain=0.1, Kvalidation=0.1, verbose=0, maxiter=5)

,best_result,regression_ensemble,inner_value_ensemble,boost_val_ens,tree_val_ens
prec@5,0.265658,0.285211,0.304566,0.317171,0.349231
1recal@5,0.737965,0.747146,0.760546,0.785112,0.814640
NDCG@5,0.271330,0.298829,0.314069,0.330188,0.363540
MAP@5,0.432628,0.469760,0.476026,0.501023,0.536181


In [29]:
netflix_ensemble_experiment(ml100k, create_list_cf(ml100k_shape)[1:5], list_ensemble, Ktrain=0.2, Kvalidation=0.1, verbose=1, maxiter=5)

0 0 [ 0.28312655  0.66129032  0.31122451  0.47448994]
0 1 [ 0.30198511  0.74689826  0.30708344  0.45899676]
0 2 [ 0.34764268  0.81885856  0.35178775  0.51665288]
0 3 [ 0.3         0.77047146  0.29314929  0.43823752]
0 4 [ 0.37295285  0.83126551  0.39198969  0.56528639]
0 5 [ 0.38957816  0.82382134  0.39621097  0.54021402]
0 6 [ 0.32332506  0.79404467  0.32910773  0.49076716]
0 7 [ 0.39478908  0.83870968  0.39405856  0.53532534]
1 0 [ 0.28064516  0.68176179  0.30430307  0.47113231]
1 1 [ 0.30136476  0.74689826  0.30803539  0.46406293]
1 2 [ 0.34540943  0.82133995  0.35088009  0.51578181]
1 3 [ 0.29913151  0.77667494  0.29969983  0.45611215]
1 4 [ 0.36575682  0.83808933  0.38443757  0.56224238]
1 5 [ 0.38362283  0.83622829  0.39270079  0.55015595]
1 6 [ 0.33325062  0.808933    0.33954008  0.50283895]
1 7 [ 0.42952854  0.87220844  0.43824681  0.59177437]
2 0 [ 0.28858561  0.69892473  0.3127589   0.48266474]
2 1 [ 0.30430108  0.75144748  0.31337982  0.47481102]
2 2 [ 0.34913151  0.82588916

,best_result,regression_ensemble,inner_value_ensemble,boost_val_ens,tree_val_ens
prec@5,0.347097,0.365409,0.386055,0.378759,0.447990
1recal@5,0.824318,0.835732,0.839454,0.842432,0.888089
NDCG@5,0.351406,0.383262,0.396607,0.388598,0.461808
MAP@5,0.513993,0.558296,0.557065,0.552341,0.622600


In [30]:
netflix_ensemble_experiment(epinion, create_list_cf(epinion_shape)[1:5], list_ensemble, Ktrain=0.1, Kvalidation=0.1, verbose=0, maxiter=3)

,best_result,regression_ensemble,inner_value_ensemble,boost_val_ens,tree_val_ens
prec@5,0.125085,0.074718,0.075414,0.127037,0.127159
1recal@5,0.422247,0.288309,0.289747,0.429436,0.430269
NDCG@5,0.132230,0.081194,0.082103,0.134896,0.134647
MAP@5,0.244243,0.164799,0.166187,0.249561,0.248770


In [31]:
netflix_ensemble_experiment(epinion, create_list_cf(epinion_shape)[1:5], list_ensemble, Ktrain=0.2, Kvalidation=0.1, verbose=0, maxiter=3)

,best_result,regression_ensemble,inner_value_ensemble,boost_val_ens,tree_val_ens
prec@5,0.170140,0.097783,0.110102,0.174014,0.171986
1recal@5,0.517367,0.355959,0.384185,0.529625,0.522966
NDCG@5,0.179068,0.102070,0.118748,0.183528,0.180995
MAP@5,0.308853,0.194671,0.225727,0.317287,0.312207


In [32]:
netflix_ensemble_experiment(ml1m, create_list_cf(ml1m_shape)[1:5], list_ensemble, Ktrain=0.1, Kvalidation=0.1, verbose=0, maxiter=3)

,best_result,regression_ensemble,inner_value_ensemble,boost_val_ens,tree_val_ens
prec@5,0.285253,0.236992,0.309485,0.351126,0.335580
1recal@5,0.726377,0.591698,0.707815,0.763621,0.743798
NDCG@5,0.293531,0.246130,0.322833,0.368761,0.351395
MAP@5,0.448811,0.375218,0.474157,0.527906,0.507440


In [33]:
netflix_ensemble_experiment(ml1m, create_list_cf(ml1m_shape)[1:5], list_ensemble, Ktrain=0.2, Kvalidation=0.1, verbose=0, maxiter=3)

,best_result,regression_ensemble,inner_value_ensemble,boost_val_ens,tree_val_ens
prec@5,0.377017,0.310242,0.377509,0.441088,0.438638
1recal@5,0.818466,0.686009,0.767766,0.838469,0.836847
NDCG@5,0.386230,0.318862,0.389958,0.458320,0.453804
MAP@5,0.541651,0.453819,0.536283,0.610640,0.603992


In [34]:
netflix_ensemble_experiment(slashdot, create_list_cf(slashdot_shape)[1:5], list_ensemble, Ktrain=0.1, Kvalidation=0.1, verbose=0, maxiter=3)

,best_result,regression_ensemble,inner_value_ensemble,boost_val_ens,tree_val_ens
prec@5,0.070862,0.036871,0.038606,0.071834,0.071596
1recal@5,0.255625,0.161137,0.166714,0.259296,0.259296
NDCG@5,0.076569,0.040013,0.041458,0.077512,0.077082
MAP@5,0.148273,0.087508,0.089294,0.150235,0.149272


In [35]:
netflix_ensemble_experiment(slashdot, create_list_cf(slashdot_shape)[1:5], list_ensemble, Ktrain=0.2, Kvalidation=0.1, verbose=0, maxiter=3)

,best_result,regression_ensemble,inner_value_ensemble,boost_val_ens,tree_val_ens
prec@5,0.098551,0.051783,0.056817,0.098427,0.098045
1recal@5,0.330282,0.214340,0.230931,0.330616,0.330950
NDCG@5,0.105175,0.054439,0.059734,0.105594,0.105629
MAP@5,0.193620,0.112115,0.122089,0.195545,0.196683


In [38]:
list_ensemble = [
    rankensemble.tree_val_ens(rankingmeasure.get_prec_K, [[3, 0],[1,2]], num_weights=11),
    rankensemble.tree_val_ens(rankingmeasure.get_one_recal_K, [[3, 0],[1,2]], num_weights=11),
    rankensemble.tree_val_ens(rankingmeasure.get_MAP, [[3, 0],[1,2]], num_weights=11),
    rankensemble.tree_val_ens(rankingmeasure.get_NDCG, [[3, 0],[1,2]], num_weights=11)
]

In [39]:
netflix_ensemble_experiment(ml100k, create_list_cf(ml100k_shape)[1:5], list_ensemble, Ktrain=0.1, Kvalidation=0.1, verbose=1, maxiter=3)

0 0 [ 0.22332506  0.61538462  0.24058859  0.39543872]
0 1 [ 0.21612903  0.63399504  0.22471367  0.38050386]
0 2 [ 0.27791563  0.73573201  0.28724598  0.44967776]
0 3 [ 0.182134    0.58808933  0.18348547  0.31746623]
0 4 [ 0.34342432  0.80769231  0.35168185  0.513191  ]
0 5 [ 0.33746898  0.79776675  0.34567404  0.50535739]
0 6 [ 0.35235732  0.80397022  0.36339672  0.52675248]
0 7 [ 0.35235732  0.80397022  0.36339672  0.52675248]
1 0 [ 0.22704715  0.62655087  0.24165509  0.39345878]
1 1 [ 0.21724566  0.63833747  0.22566895  0.37891594]
1 2 [ 0.26761787  0.73387097  0.27435782  0.43690895]
1 3 [ 0.18411911  0.58498759  0.18479258  0.31469965]
1 4 [ 0.34367246  0.80707196  0.35214689  0.51401124]
1 5 [ 0.33176179  0.79280397  0.34004437  0.50103822]
1 6 [ 0.3439206   0.80024814  0.35172154  0.51168752]
1 7 [ 0.3439206   0.80024814  0.35172154  0.51168752]
2 0 [ 0.22828784  0.62365591  0.24232948  0.3938442 ]
2 1 [ 0.21488834  0.63482217  0.22443837  0.37886109]
2 2 [ 0.26311001  0.72870141

,best_result,tree_val_ens,tree_val_ens,tree_val_ens,tree_val_ens
prec@5,0.263110,0.342349,0.334160,0.347891,0.348304
1recal@5,0.728701,0.805211,0.801075,0.807692,0.807692
NDCG@5,0.269182,0.352850,0.344639,0.357288,0.358078
MAP@5,0.429844,0.518629,0.511584,0.521104,0.522668


In [40]:
netflix_ensemble_experiment(ml100k, create_list_cf(ml100k_shape)[1:5], list_ensemble, Ktrain=0.2, Kvalidation=0.1, verbose=0, maxiter=3)

,best_result,tree_val_ens,tree_val_ens,tree_val_ens,tree_val_ens
prec@5,0.351447,0.440199,0.444003,0.445988,0.453267
1recal@5,0.836228,0.893714,0.893714,0.889578,0.896195
NDCG@5,0.360177,0.450945,0.457084,0.460962,0.468127
MAP@5,0.529090,0.614101,0.619340,0.622686,0.630066


In [41]:
netflix_ensemble_experiment(epinion, create_list_cf(epinion_shape)[1:5], list_ensemble, Ktrain=0.1, Kvalidation=0.1, verbose=0, maxiter=3)

,best_result,tree_val_ens,tree_val_ens,tree_val_ens,tree_val_ens
prec@5,0.124495,0.124586,0.124601,0.126432,0.126054
1recal@5,0.419296,0.422550,0.422626,0.426182,0.426182
NDCG@5,0.131244,0.130698,0.130795,0.133830,0.133107
MAP@5,0.241765,0.241910,0.242172,0.247452,0.246365


In [42]:
netflix_ensemble_experiment(epinion, create_list_cf(epinion_shape)[1:5], list_ensemble, Ktrain=0.2, Kvalidation=0.1, verbose=0, maxiter=3)

,best_result,tree_val_ens,tree_val_ens,tree_val_ens,tree_val_ens
prec@5,0.170170,0.170655,0.170261,0.171426,0.171078
1recal@5,0.519107,0.525766,0.525085,0.528036,0.526523
NDCG@5,0.178672,0.178919,0.178419,0.180642,0.180292
MAP@5,0.307888,0.311483,0.310681,0.315460,0.314149


In [43]:
netflix_ensemble_experiment(ml1m, create_list_cf(ml1m_shape)[1:5], list_ensemble, Ktrain=0.1, Kvalidation=0.1, verbose=0, maxiter=3)

,best_result,tree_val_ens,tree_val_ens,tree_val_ens,tree_val_ens
prec@5,0.287884,0.318183,0.335916,0.354971,0.347666
1recal@5,0.728059,0.716586,0.738752,0.765964,0.752748
NDCG@5,0.296990,0.331574,0.351323,0.371440,0.364689
MAP@5,0.453680,0.482110,0.506411,0.529045,0.522080


In [44]:
netflix_ensemble_experiment(ml1m, create_list_cf(ml1m_shape)[1:5], list_ensemble, Ktrain=0.2, Kvalidation=0.1, verbose=0, maxiter=3)

,best_result,tree_val_ens,tree_val_ens,tree_val_ens,tree_val_ens
prec@5,0.377281,0.440992,0.441233,0.439346,0.438457
1recal@5,0.819066,0.836968,0.838229,0.840091,0.838770
NDCG@5,0.387390,0.457159,0.456664,0.455191,0.453829
MAP@5,0.544753,0.608062,0.607252,0.607510,0.605531


In [45]:
netflix_ensemble_experiment(slashdot, create_list_cf(slashdot_shape)[1:5], list_ensemble, Ktrain=0.1, Kvalidation=0.1, verbose=0, maxiter=3)

,best_result,tree_val_ens,tree_val_ens,tree_val_ens,tree_val_ens
prec@5,0.070986,0.070614,0.070881,0.070681,0.070614
1recal@5,0.255912,0.255196,0.255625,0.255053,0.255196
NDCG@5,0.076179,0.076152,0.076223,0.076535,0.076152
MAP@5,0.147365,0.147990,0.147661,0.149136,0.147990


In [46]:
netflix_ensemble_experiment(slashdot, create_list_cf(slashdot_shape)[1:5], list_ensemble, Ktrain=0.2, Kvalidation=0.1, verbose=0, maxiter=3)

,best_result,tree_val_ens,tree_val_ens,tree_val_ens,tree_val_ens
prec@5,0.099190,0.099485,0.099676,0.099476,0.099619
1recal@5,0.334287,0.335383,0.335669,0.334811,0.334859
NDCG@5,0.105836,0.106319,0.106761,0.106400,0.106467
MAP@5,0.195234,0.196584,0.197287,0.196464,0.196376
